## 使用套件

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from urllib.parse import quote
import re
import requests
import sys
import os
import time
import json
import html2text
import pandas as pd
import undetected_chromedriver as uc

#### URL列表

In [37]:
# url列表
url_list = [
    "https://www.google.com.tw/maps/place/%E9%AD%9A%E3%81%AE%E6%A3%A7(%E9%AD%9A%E4%B9%8B%E6%A3%A7)/data=!4m7!3m6!1s0x3442a94844003971:0x4aeef21e060b771a!8m2!3d25.0712473!4d121.5159715!16s%2Fg%2F11c1rfpcpf!19sChIJcTkAREipQjQRGncLBh7y7ko?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%95%83%E8%8C%84%E5%A6%B9%E5%B0%8F%E5%90%83%E5%BA%97/data=!4m7!3m6!1s0x3442a949b3b63957:0x7b22d945beadce01!8m2!3d25.0713438!4d121.5158121!16s%2Fg%2F1td2dls1!19sChIJVzm2s0mpQjQRAc6tvkXZIns?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%98%BF%E4%BB%81%E7%82%92%E9%A3%AF/data=!4m7!3m6!1s0x3442a9497396caf1:0x93d2d5a866d16d66!8m2!3d25.073453!4d121.517429!16s%2Fg%2F1pzt7hg4r!19sChIJ8cqWc0mpQjQRZm3RZqjV0pM?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E2%80%A2%E5%A3%B9%E6%9F%92%E9%A3%9F%E5%A0%82%E2%80%A2/data=!4m7!3m6!1s0x3442a9b2d94f03c7:0xd5f4285c70a63d22!8m2!3d25.0653876!4d121.5159267!16s%2Fg%2F11q8tjwtqf!19sChIJxwNP2bKpQjQRIj2mcFwo9NU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B7%B4%E5%85%AB%E5%85%AB%E5%B0%8F%E5%90%83%E5%BA%97/data=!4m7!3m6!1s0x3442a9486a777257:0xafcdc9f1eb101e26!8m2!3d25.0709175!4d121.5157808!16s%2Fg%2F1pzqllyzs!19sChIJV3J3akipQjQRJh4Q6_HJza8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%90%B5%E6%9D%BF%E5%9F%8E%E7%8F%BE%E7%82%92%EF%BC%88%E5%A4%A7%E9%BE%8D%E5%BA%97%EF%BC%89/data=!4m7!3m6!1s0x3442a9486a7ee9c3:0x7d7610e42293b655!8m2!3d25.0709856!4d121.5158247!16s%2Fg%2F11ckv6w1qc!19sChIJw-l-akipQjQRVbaTIuQQdn0?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%9D%8E%E8%A8%98%E9%9B%9E%E7%AA%A9%2F%E5%A4%A7%E5%90%8C%E7%BE%8E%E9%A3%9F%2F%E5%A4%A7%E5%90%8C%E9%9B%9E%E8%82%89%2F%E5%A4%A7%E5%90%8C%E4%BE%BF%E7%95%B6%2F%E5%A4%A7%E5%90%8C%E5%B0%8F%E5%90%83%2F%E5%A4%A7%E5%90%8C%E9%A4%90%E5%BB%B3%2F%E5%A4%A7%E5%90%8C%E6%94%BE%E5%B1%B1%E9%9B%9E/data=!4m7!3m6!1s0x3442a9961212d99d:0x5f9e8d42246fa31f!8m2!3d25.063948!4d121.5160627!16s%2Fg%2F11j97jm46q!19sChIJndkSEpapQjQRH6NvJEKNnl8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E7%89%9B%E7%89%9B%E6%8E%92%E9%A4%A8/data=!4m7!3m6!1s0x3442a9486b9771e1:0xc5ed7b719909670d!8m2!3d25.0706528!4d121.5157755!16s%2Fg%2F12m9kx30r!19sChIJ4XGXa0ipQjQRDWcJmXF77cU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97%E9%A6%99%E8%8F%87%E8%82%89%E7%B2%A5/data=!4m7!3m6!1s0x3442a9484ecac05d:0xd5c8478a939aa4c2!8m2!3d25.0738802!4d121.5140394!16s%2Fg%2F1vg_9km9!19sChIJXcDKTkipQjQRwqSak4pHyNU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%BB%A3%E6%9D%B1%E7%B2%A5/data=!4m7!3m6!1s0x3442a960f1a68bf5:0xbff068f555ca3bfe!8m2!3d25.0667749!4d121.5158718!16s%2Fg%2F11p_1pnfxc!19sChIJ9Yum8WCpQjQR_jvKVfVo8L8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%9B%84%E9%A6%99%E9%9B%9E%E7%BF%85/data=!4m7!3m6!1s0x3442a9484eccef35:0x6c2a5228f2e31fb8!8m2!3d25.0706504!4d121.5158399!16s%2Fg%2F12hqq7355!19sChIJNe_MTkipQjQRuB_j8ihSKmw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%B3%92%E8%82%89%E7%BE%B9-%E7%B1%B3%E7%B2%89%E6%B9%AF/data=!4m7!3m6!1s0x3442a936273a7c8f:0xb51adae249d7e51b!8m2!3d25.073688!4d121.5140449!16s%2Fg%2F12vtjq153!19sChIJj3w6JzapQjQRG-XXSeLaGrU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%80%81%E5%9C%B0%E6%96%B9%E7%89%9B%E8%82%89%E9%BA%B5%E9%A3%9F%E9%A4%A8/data=!4m7!3m6!1s0x3442a9464e6f02cd:0x79f08aa8dae79fd7!8m2!3d25.0673476!4d121.5173978!16s%2Fg%2F1tyks1s4!19sChIJzQJvTkapQjQR15_n2qiK8Hk?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%82%B8G%E8%A1%8C%E5%AE%B6-%E5%A4%A7%E9%BE%8D%E5%B3%92%E5%BA%97%EF%BC%88%E7%82%B8%E9%9B%9E%E8%A1%8C%E5%AE%B6.%E7%BE%8E%E5%BC%8F%E7%82%B8%E9%9B%9E.%E5%B9%B3%E5%83%B9%E7%BE%8E%E9%A3%9F%EF%BC%89/data=!4m7!3m6!1s0x3442a9c099ca6745:0xf548dbfd05e356a9!8m2!3d25.0744639!4d121.5155667!16s%2Fg%2F11pwyv2cnr!19sChIJRWfKmcCpQjQRqVbjBf3bSPU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E4%B8%89%E5%A4%A7%E7%89%9B%E8%82%89%E9%BA%B5/data=!4m7!3m6!1s0x3442a940c4412d1b:0x29418d04468a6eea!8m2!3d25.0633697!4d121.5160486!16s%2Fg%2F11cmsd7549!19sChIJGy1BxECpQjQR6m6KRgSNQSk?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%96%8B%E7%87%88%E6%89%BE%E9%A4%90/data=!4m7!3m6!1s0x3442a940cf6f0e55:0x7650fed66f7259fe!8m2!3d25.0635658!4d121.5161075!16s%2Fg%2F11bz_xz1y7!19sChIJVQ5vz0CpQjQR_llyb9b-UHY?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%AB%98%E9%9B%84%E7%87%92%E8%82%89%E9%A3%AF/data=!4m7!3m6!1s0x3442a9484f327cd5:0x715af1caedb9bc2!8m2!3d25.0717682!4d121.5159579!16s%2Fg%2F1w0j1x8f!19sChIJ1XwyT0ipQjQRwpvbrhyvFQc?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%95%BC%E8%92%94%E8%8D%B3%E7%95%99/data=!4m7!3m6!1s0x3442a984a1ad01ef:0x8f9a0faf9b6aa889!8m2!3d25.066367!4d121.5163167!16s%2Fg%2F11pklx0km4!19sChIJ7wGtoYSpQjQRiahqm68Pmo8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E6%B2%99%E5%A8%81%E7%91%AA/data=!4m7!3m6!1s0x3442a9484682d0a1:0x611d055a44d5e176!8m2!3d25.0713296!4d121.5160148!16s%2Fg%2F11gdthdf7j!19sChIJodCCRkipQjQRduHVRFoFHWE?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%AC%AC%E4%B8%80%E5%9C%9F%E9%B5%9D%E8%82%89/data=!4m7!3m6!1s0x3442a9484e9f6421:0x2db720ae76aa617!8m2!3d25.0719177!4d121.5160689!16s%2Fg%2F1tmcjy5h!19sChIJIWSfTkipQjQRF6Zq5wpy2wI?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B0%8F%E6%9E%97%E9%BA%B5%E9%A3%9F%E5%BA%97/data=!4m7!3m6!1s0x3442a9464d47ab55:0xf7afcb147d969f96!8m2!3d25.0674022!4d121.517082!16s%2Fg%2F1thvdmmp!19sChIJVatHTUapQjQRlp-WfRTLr_c?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%B9%AF%E7%AB%BA%E9%BA%B5%E9%A3%9F%E9%A4%A8/data=!4m7!3m6!1s0x3442a9a8b23a1dd7:0x99f8dcb085c6adda!8m2!3d25.0689414!4d121.5158265!16s%2Fg%2F11sy3hzz5d!19sChIJ1x06sqipQjQR2q3GhbDc-Jk?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%9F%8E%E8%87%AA%E5%8A%A9%E9%A4%90/data=!4m7!3m6!1s0x3442a94844273e53:0xda30e88f96e32d3c!8m2!3d25.0713297!4d121.5160147!16s%2Fg%2F11k5n4nlws!19sChIJUz4nREipQjQRPC3jlo_oMNo?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%8F%B0%E5%8D%97%E6%9D%B1%E5%B1%B1%E9%B4%A8%E9%A0%AD/data=!4m7!3m6!1s0x3442a94848faaf21:0x68fd3154991b2e66!8m2!3d25.0717324!4d121.5157837!16s%2Fg%2F1pzxgftkl!19sChIJIa_6SEipQjQRZi4bmVQx_Wg?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E8%82%A5%E7%BE%B4%E7%87%92%E7%83%A4%E7%BE%8A%E8%82%89%E8%BD%89%E5%9E%8B%EF%BC%88%E5%8F%A4%E6%97%A9%E5%91%B3%E9%BA%B5%E5%BA%97%EF%BC%89/data=!4m7!3m6!1s0x3442a941e07acd9f:0x1c67d0e7d03916f0!8m2!3d25.0699618!4d121.5158324!16s%2Fg%2F1tfbqk_5!19sChIJn8164EGpQjQR8BY50OfQZxw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%B0%A1%E5%AE%B6%E5%A4%A7%E9%BE%8D%E5%B3%92%E8%82%89%E5%9C%93/data=!4m7!3m6!1s0x3442a947dcd32999:0x14f72d848cf5ffc6!8m2!3d25.0689089!4d121.5158355!16s%2Fg%2F1thktzct!19sChIJmSnT3EepQjQRxv_1jIQt9xQ?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E7%9C%BE%E6%B5%B7%E9%AE%AE%E7%B2%A5/data=!4m7!3m6!1s0x3442a948461505d5:0x7d2f1bf3c8c2738e!8m2!3d25.0715451!4d121.5158356!16s%2Fg%2F11c585f_07!19sChIJ1QUVRkipQjQRjnPCyPMbL30?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%9B%99%E9%80%A3+%E6%96%87%E8%A8%98%E8%8A%B1%E6%9E%9D%E7%84%BF/data=!4m7!3m6!1s0x3442a9486a4343ad:0x554a90f115bbeb6d!8m2!3d25.0708232!4d121.5158137!16s%2Fg%2F11h05d9ky!19sChIJrUNDakipQjQRbeu7FfGQSlU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%80%81%E8%94%A1%E6%B0%B4%E7%85%8E%E5%8C%85+%E5%A4%A7%E9%BE%8D%E5%89%B5%E5%A7%8B%E5%BA%97/data=!4m7!3m6!1s0x3442a94849047ebf:0xd9da1166fe37032b!8m2!3d25.0717855!4d121.5157835!16s%2Fg%2F1tdnbysx!19sChIJv34ESUipQjQRKwM3_mYR2tk?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%90%89%E7%A5%A5%E7%B4%A0%E9%A3%9F/data=!4m7!3m6!1s0x3442a947df697b59:0x9fbb9c96f6576125!8m2!3d25.069554!4d121.515843!16s%2Fg%2F11ycqtyq_!19sChIJWXtp30epQjQRJWFX9pacu58?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%87%8D%E6%85%B6%E8%B1%86%E6%BC%BF%E7%82%B8%E8%9B%8B%E9%A4%85/data=!4m7!3m6!1s0x3442a949f1cbd53d:0x82d33c5d6552191!8m2!3d25.0744676!4d121.5150754!16s%2Fg%2F1tgbygbb!19sChIJPdXL8UmpQjQRkSFV1sUzLQg?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/272%E6%97%A9%E5%8D%88%E9%A4%90/data=!4m7!3m6!1s0x3442a9486a4e0eb7:0xd3ef776e6760c56c!8m2!3d25.0707833!4d121.5158438!16s%2Fg%2F11_rqbm6d!19sChIJtw5OakipQjQRbMVgZ25379M?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%90%89%E5%88%A9+%E7%89%9B%E8%82%89%E9%BA%B5.%E6%B0%B4%E9%A4%83/data=!4m7!3m6!1s0x3442a94841c7b893:0x408d8c199754d502!8m2!3d25.0709993!4d121.5158148!16s%2Fg%2F1whdg12t!19sChIJk7jHQUipQjQRAtVUlxmMjUA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%B3%92%E9%AD%9A%E4%B8%B8/data=!4m7!3m6!1s0x3442a9362421311f:0x61f33430562c9d4b!8m2!3d25.0734356!4d121.5135913!16s%2Fg%2F11t28c2n95!19sChIJHzEhJDapQjQRS50sVjA082E?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%8F%81%E9%81%93%E8%A1%97%E6%89%8B%E5%B7%A5%E8%9A%B5%E4%BB%94%E9%BA%B5%E7%B7%9A/data=!4m7!3m6!1s0x3442a90bb3793569:0xe6fc69499648c436!8m2!3d25.0666387!4d121.5158689!16s%2Fg%2F11r9cbrt2q!19sChIJaTV5swupQjQRNsRIlklp_OY?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E4%B8%89%E5%AA%BD%E8%87%AD%E8%87%AD%E9%8D%8B/data=!4m7!3m6!1s0x3442a94845d67159:0xcdd9729e4d68eff0!8m2!3d25.0713893!4d121.515964!16s%2Fg%2F12ml2xz43!19sChIJWXHWRUipQjQR8O9oTZ5y2c0?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97271%E8%99%9F%E9%BB%83%E9%87%91%E5%9C%B0%E7%93%9CQQ%E7%90%83/data=!4m7!3m6!1s0x3442a948494fed53:0xa930d04c59e870aa!8m2!3d25.0717907!4d121.5159395!16s%2Fg%2F11f5ltv4f9!19sChIJU-1PSUipQjQRqnDoWUzQMKk?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%87%91%E5%A5%BD%E5%91%B7%E5%A5%BD%E5%90%83%E9%9B%9E%E8%82%89%E9%A3%AF/data=!4m7!3m6!1s0x3442a94837cccf93:0x4852b4f865688766!8m2!3d25.0719344!4d121.5155479!16s%2Fg%2F11c58kqszt!19sChIJk8_MN0ipQjQRZodoZfi0Ukg?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%96%B0%E5%8F%A3%E5%91%B3%E9%B9%BD%E9%85%A5%E9%9B%9E/data=!4m7!3m6!1s0x3442a948490b44d5:0xacf14da492652ae6!8m2!3d25.0718524!4d121.5157725!16s%2Fg%2F11b63vwxf_!19sChIJ1UQLSUipQjQR5iplkqRN8aw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%85%A8%E9%BE%8D%E5%81%A5%E5%BA%B7%E7%B4%A0%E9%A3%9F/data=!4m7!3m6!1s0x3442a94841781b77:0xc891d1a14622a9ed!8m2!3d25.0711348!4d121.5158126!16s%2Fg%2F11xdm8b8y!19sChIJdxt4QUipQjQR7akiRqHRkcg?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B0%9A%E5%93%81%E9%A6%99%E6%97%A9%E9%A4%90%E5%BA%97%EF%BC%88%E5%A4%A7%E9%BE%8D%E5%BA%97%EF%BC%89/data=!4m7!3m6!1s0x3442a940c46becb3:0x4549db9c18e935cd!8m2!3d25.0641424!4d121.5160745!16s%2Fg%2F11f5bp6cdx!19sChIJs-xrxECpQjQRzTXpGJzbSUU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B0%8F%E9%87%8C%E5%AD%90%E6%B5%B7%E9%AE%AE/data=!4m7!3m6!1s0x3442a949c4c78fe1:0x448b414b7582b95e!8m2!3d25.0730409!4d121.5145527!16s%2Fg%2F1trtr_wn!19sChIJ4Y_HxEmpQjQRXrmCdUtBi0Q?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%9B%B2%E7%A5%A5%E9%BA%B5%E9%A3%9F/data=!4m7!3m6!1s0x3442a9470e552ce5:0xff2a099c822aca5!8m2!3d25.0663676!4d121.5161888!16s%2Fg%2F11c1v8kj45!19sChIJ5SxVDkepQjQRpawiyJmg8g8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E4%BA%AC%E5%93%81%E5%BB%A3%E6%9D%B1%E7%B2%A5/data=!4m7!3m6!1s0x3442a9481f03acab:0xd153a04c8eab5c36!8m2!3d25.0711215!4d121.5158112!16s%2Fg%2F11ggvwmlq_!19sChIJq6wDH0ipQjQRNlyrjkygU9E?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%BD%AE%E5%91%B3%E6%B1%BA%E2%80%A7%E6%B9%AF%E6%BB%B7%E5%B0%88%E9%96%80%E5%BA%97+%E5%8F%B0%E5%8C%97%E5%A4%A7%E9%BE%8D%E5%88%86%E7%A4%BE/data=!4m7!3m6!1s0x3442a98a90bf2155:0x6a49f3fe11710d8c!8m2!3d25.07191!4d121.5148745!16s%2Fg%2F11hfswjl2c!19sChIJVSG_kIqpQjQRjA1xEf7zSWo?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%94%90%E5%B1%B1%E5%A9%86%E5%8F%A4%E6%97%A9%E5%91%B3%E7%82%B8%E8%82%89/data=!4m7!3m6!1s0x3442a95e45d79593:0xaac2d897ccf23971!8m2!3d25.0718524!4d121.5152422!16s%2Fg%2F11qydjpjcc!19sChIJk5XXRV6pQjQRcTnyzJfYwqo?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E6%97%A9%E5%8D%88%E9%A4%90/data=!4m7!3m6!1s0x3442a96b65136809:0x234759490ef0cbd1!8m2!3d25.0745646!4d121.5148935!16s%2Fg%2F11g7272m18!19sChIJCWgTZWupQjQR0cvwDklZRyM?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%B3%92%E7%86%8A%E6%9C%83%E9%95%B7%E6%B2%B9%E9%A3%AF/data=!4m7!3m6!1s0x3442a949e5751d83:0x3c919cdd2c1ef291!8m2!3d25.0743426!4d121.5148633!16s%2Fg%2F1tmk660w!19sChIJgx115UmpQjQRkfIeLN2ckTw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97%E5%A4%9C%E5%B8%82-%E5%BB%9F%E5%8F%A3%E7%9A%84%E9%A3%AF/data=!4m7!3m6!1s0x3442a94846a78377:0xbdabc43aec95ec4c!8m2!3d25.0711406!4d121.5159757!16s%2Fg%2F11bzypqg58!19sChIJd4OnRkipQjQRTOyV7DrEq70?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%9E%9C%E7%84%B6%E6%96%87%E5%89%B5/data=!4m7!3m6!1s0x3442a94a1bfa68c7:0x8bb33009af6ec818!8m2!3d25.0750043!4d121.5151788!16s%2Fg%2F11c1vpq6pc!19sChIJx2j6G0qpQjQRGMhurwkws4s?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%B3%92%E4%B8%80%E7%94%9F%E6%87%B8%E5%91%BD%E9%BB%91%E7%99%BD%E5%88%87/data=!4m7!3m6!1s0x3442a9682d19f8b5:0x8a8c94ec7b4a8381!8m2!3d25.0741392!4d121.5142351!16s%2Fg%2F11t0p9snt5!19sChIJtfgZLWipQjQRgYNKe-yUjIo?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%A2%85%E6%BB%BF%E7%BE%8E%E9%A3%9F/data=!4m7!3m6!1s0x3442a94905d34bdb:0xe63257a567762c48!8m2!3d25.0723119!4d121.5173322!16s%2Fg%2F1tj98py3!19sChIJ20vTBUmpQjQRSCx2Z6VXMuY?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%AE%9A%E8%B3%9E%E9%BA%B5%E5%B1%8B/data=!4m7!3m6!1s0x3442a9f71788f239:0x9e246e8a872e3ec6!8m2!3d25.0715184!4d121.5155836!16s%2Fg%2F11nxt5m9tv!19sChIJOfKIF_epQjQRxj4uh4puJJ4?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%BC%98%E7%88%BA%E6%BC%A2%E5%A0%A1/data=!4m7!3m6!1s0x3442a940cee4b527:0xb25b12aba04561e8!8m2!3d25.0637497!4d121.5160663!16s%2Fg%2F1tdkdb50!19sChIJJ7XkzkCpQjQR6GFFoKsSW7I?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%BE%B7%E9%BE%8D%E9%A3%B2%E9%A3%9F%E5%BA%97/data=!4m7!3m6!1s0x3442a9484ed469e7:0xe6393536716780fd!8m2!3d25.071758!4d121.5159609!16s%2Fg%2F1w97s2pv!19sChIJ52nUTkipQjQR_YBncTY1OeY?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%AF%B6%E5%B8%AB%E5%82%85%E7%A7%81%E6%88%BF%E7%89%9B%E8%82%89%E9%BA%B5/data=!4m7!3m6!1s0x3442a9498d380abd:0x4020540826fb59b8!8m2!3d25.0744511!4d121.5156486!16s%2Fg%2F1tctc92g!19sChIJvQo4jUmpQjQRuFn7JghUIEA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%AC%AC%E4%B8%80%E8%8C%97%E7%8B%80%E5%85%83%E7%B3%95/data=!4m7!3m6!1s0x3442a9341d6208f1:0xf58a9ef9b4f052be!8m2!3d25.0717911!4d121.5159483!16s%2Fg%2F11swbry41y!19sChIJ8QhiHTSpQjQRvlLwtPmeivU?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%90%AC%E4%B8%96%E9%A6%99%E5%B0%8F%E7%B1%A0%E5%8C%85%E8%B1%86%E6%BC%BF%E5%BA%97/data=!4m7!3m6!1s0x3442a947bc018a67:0x10271d42aaa02209!8m2!3d25.0707233!4d121.515825!16s%2Fg%2F1tsygx7z!19sChIJZ4oBvEepQjQRCSKgqkIdJxA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%80%81%E9%BA%B5%E5%BA%97/data=!4m7!3m6!1s0x3442a938296ed9e1:0xe418d4d5ff9483de!8m2!3d25.0703643!4d121.5116522!16s%2Fg%2F11c3k6dnm5!19sChIJ4dluKTipQjQR3oOU_9XUGOQ?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/Q+Burger+%E5%A4%A7%E5%90%8C%E5%A4%A7%E9%BE%8D%E5%BA%97/data=!4m7!3m6!1s0x3442a91be64e4aed:0xdd7b73af08567716!8m2!3d25.0677692!4d121.5158121!16s%2Fg%2F11k5n4w9gw!19sChIJ7UpO5hupQjQRFndWCK9ze90?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%9C%93%E7%92%B0%E9%AD%AF%E8%82%89%E9%A3%AF%E8%82%89%E7%BE%B9%2F%E5%A4%A7%E5%90%8C%E5%8D%80%E5%B0%8F%E5%90%83+%E5%A4%A7%E5%90%8C%E5%8D%80%E7%BE%8E%E9%A3%9F+%E5%A4%A7%E5%90%8C%E5%8D%80%E6%BB%B7%E8%82%89%E9%A3%AF+%E5%9C%93%E5%B1%B1%E7%BE%8E%E9%A3%9F+%E5%9C%93%E5%B1%B1%E5%B0%8F%E5%90%83+%E5%9C%93%E5%B1%B1%E6%BB%B7%E8%82%89%E9%A3%AF/data=!4m7!3m6!1s0x3442a947dcedc067:0xa008838eb1560ca4!8m2!3d25.0693552!4d121.5158665!16s%2Fg%2F1tf6zmy6!19sChIJZ8Dt3EepQjQRpAxWsY6DCKA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%B2%B9%E6%A2%9D%E8%8A%B1%E7%94%9F%E6%B9%AF/data=!4m7!3m6!1s0x3442a948462b05d9:0xc265f35f61433a5e!8m2!3d25.0716402!4d121.5157756!16s%2Fg%2F1pzpzxfmf!19sChIJ2QUrRkipQjQRXjpDYV_zZcI?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/5%E5%91%B3%E8%87%AD%E8%87%AD%E9%8D%8B+%E5%A4%A7%E9%BE%8D%E5%BA%97/data=!4m7!3m6!1s0x3442a9484884cbf5:0xd6f828a992c31b0c!8m2!3d25.0716643!4d121.5157756!16s%2Fg%2F12hlxnqxr!19sChIJ9cuESEipQjQRDBvDkqko-NY?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B0%8F%E6%BC%A2%E5%A0%A1%E6%97%A9%E9%A4%90/data=!4m7!3m6!1s0x3442a9470fb75225:0x3ca646b41d154470!8m2!3d25.066342!4d121.5161807!16s%2Fg%2F11cs0f8859!19sChIJJVK3D0epQjQRcEQVHbRGpjw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/Camilla+Breakfast/data=!4m7!3m6!1s0x3442a9464ddf3d77:0x485404f565f54732!8m2!3d25.0638305!4d121.5159185!16s%2Fg%2F11c555_ngv!19sChIJdz3fTUapQjQRMkf1ZfUEVEg?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E4%BE%BF%E7%95%B6%E5%8F%8A%E5%8F%A4%E6%97%A9%E5%91%B3%E4%BB%80%E9%8C%A6%E9%BA%B5/data=!4m7!3m6!1s0x3442a9c2984fd6b5:0x5151fe8d486f08fd!8m2!3d25.0699618!4d121.5158324!16s%2Fg%2F11t1s9xjbv!19sChIJtdZPmMKpQjQR_QhvSI3-UVE?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97%E5%A4%9C%E5%B8%82/data=!4m7!3m6!1s0x3442a9486bb0653d:0xd9a3dd4cadd71cc6!8m2!3d25.0714662!4d121.5159014!16s%2Fg%2F1q62mmlql!19sChIJPWWwa0ipQjQRxhzXrUzdo9k?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/MILI+%E7%B1%B3%E9%BA%97%E5%BB%9A%E6%88%BF+%E5%A4%A7%E9%BE%8D%E5%BA%97/data=!4m7!3m6!1s0x3442a98d0753494d:0xcd0182dc1a61de6d!8m2!3d25.0705968!4d121.5170922!16s%2Fg%2F11mwsc84h5!19sChIJTUlTB42pQjQRbd5hGtyCAc0?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%BE%8E%E5%8F%88%E5%A0%A1/data=!4m7!3m6!1s0x3442a940c8b225ef:0x72bd57c21c697e80!8m2!3d25.063676!4d121.5159314!16s%2Fg%2F11fxzrnt3s!19sChIJ7yWyyECpQjQRgH5pHMJXvXI?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%88%89%E4%BA%BA%E8%B1%86%E8%8A%B1/data=!4m7!3m6!1s0x3442a91a35210f43:0x9046aced378b977d!8m2!3d25.0749794!4d121.5151895!16s%2Fg%2F11hjbhtrgf!19sChIJQw8hNRqpQjQRfZeLN-2sRpA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/JOMO+Coffee+Roaster+%E5%A4%A7%E9%BE%8D%E5%BA%97/data=!4m7!3m6!1s0x3442a9bb1a3c5ac9:0x3c4b9b4783057083!8m2!3d25.0673969!4d121.5169958!16s%2Fg%2F11j6kfxkff!19sChIJyVo8GrupQjQRg3AFg0ebSzw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/PASTA+RISOTTO/data=!4m7!3m6!1s0x3442a9363288f505:0x8b4bea904cfe6c24!8m2!3d25.0729894!4d121.5134826!16s%2Fg%2F1pzr3fnry!19sChIJBfWIMjapQjQRJGz-TJDqS4s?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%AE%9A%E5%A5%BD%E5%90%83%E5%8F%A4%E6%97%A9%E5%91%B3%E9%9B%9E%E8%9B%8B%E7%B3%95/data=!4m7!3m6!1s0x3442a937d84d6685:0x5c5667a5cea2f0bc!8m2!3d25.0744639!4d121.5155667!16s%2Fg%2F11b63vlg2t!19sChIJhWZN2DepQjQRvPCizqVnVlw?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%BE%8E%E7%8F%8D%E8%B1%86%E6%BC%BF%E6%97%A9%E9%A4%90%E5%BA%97/data=!4m7!3m6!1s0x3442a9cf67357e89:0x8f9d639d914a36e8!8m2!3d25.0665486!4d121.5170651!16s%2Fg%2F11hzmnvlrl!19sChIJiX41Z8-pQjQR6DZKkZ1jnY8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%B6%A0%E5%A2%83+Aroma/data=!4m7!3m6!1s0x3442a9110075ade5:0xecf450543da9f000!8m2!3d25.0753091!4d121.5158152!16s%2Fg%2F11jy3f9_gx!19sChIJ5a11ABGpQjQRAPCpPVRQ9Ow?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E5%8F%94%E9%A3%9F%E4%BA%8B/data=!4m7!3m6!1s0x3442a90d6d9deadf:0xd3546171991a5f4f!8m2!3d25.0736424!4d121.5174779!16s%2Fg%2F11fsbbd42j!19sChIJ3-qdbQ2pQjQRT18amXFhVNM?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97%E9%B9%B9%E6%B0%B4%E9%9B%9E+%E9%BA%BB%E8%BE%A3%E9%9B%9E/data=!4m7!3m6!1s0x3442a9484eab3089:0xb05d7b57f01397d3!8m2!3d25.0717907!4d121.5159395!16s%2Fg%2F11c548t3bw!19sChIJiTCrTkipQjQR05cT8Fd7XbA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%B2%B9%E9%A3%AF%E5%B0%8F%E5%90%83%E9%BA%B5%E5%BA%97/data=!4m7!3m6!1s0x3442a9809c226149:0x8435992f3673f89f!8m2!3d25.0665769!4d121.5158608!16s%2Fg%2F11sz95kgw7!19sChIJSWEinICpQjQRn_hzNi-ZNYQ?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%82%B3%E7%B5%B1%E9%A6%99%E6%BB%B7%E5%91%B3/data=!4m7!3m6!1s0x3442a96a743defb1:0xbe315e2ae66203e1!8m2!3d25.0717907!4d121.5159395!16s%2Fg%2F11j3t09hhp!19sChIJse89dGqpQjQR4QNi5ipeMb4?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%9D%8E%E8%A8%98%E5%B7%9D%E5%91%B3%E5%B0%8F%E5%90%83%E5%BA%97/data=!4m7!3m6!1s0x3442a947a123f9e9:0x2b7f8126feffc5c3!8m2!3d25.0671429!4d121.5158779!16s%2Fg%2F1tfk6q6f!19sChIJ6fkjoUepQjQRw8X__iaBfys?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B7%9D%E5%A9%86%E5%A8%98%E5%9B%9B%E5%B7%9D%E9%BA%BB%E8%BE%A3%E7%87%99%E5%8A%A0%E7%86%B1%E6%BB%B7%E5%91%B3/data=!4m7!3m6!1s0x3442a94875c58b39:0x69901a4e067b6415!8m2!3d25.0697339!4d121.515855!16s%2Fg%2F11c1nlslty!19sChIJOYvFdUipQjQRFWR7Bk4akGk?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B7%A8%E6%9E%97%E7%BE%8E%E8%80%8C%E7%BE%8E/data=!4m7!3m6!1s0x3442a9470c0b75fb:0xb2e040519df57a87!8m2!3d25.0666081!4d121.5158678!16s%2Fg%2F11bzlxh9z1!19sChIJ-3ULDEepQjQRh3r1nVFA4LI?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%97%A5%E5%BC%8F%E6%97%A5%E6%9C%AC%E9%A2%A8%E5%91%B3%E6%96%99%E7%90%86/data=!4m7!3m6!1s0x3442a949fae522c3:0x59db2adec3c8b166!8m2!3d25.0745633!4d121.5147437!16s%2Fg%2F11c72r1xh0!19sChIJwyLl-kmpQjQRZrHIw94q21k?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%BE%8E%E7%AB%8B%E7%BE%8E%E6%97%A9%E9%A4%90%E5%BA%97/data=!4m7!3m6!1s0x3442a947df6491b7:0x6ed4e5099d6a1a7b!8m2!3d25.0689538!4d121.5159901!16s%2Fg%2F11btvwv137!19sChIJt5Fk30epQjQRexpqnQnl1G4?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97%E9%98%BF%E5%BF%A0%E7%A2%B3%E7%83%A4%E5%BA%97/data=!4m7!3m6!1s0x3442a94706b72d69:0x5a33fadd07b396eb!8m2!3d25.066488!4d121.516655!16s%2Fg%2F11bxjlvfzc!19sChIJaS23BkepQjQR65azB936M1o?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E6%BC%81%E9%99%B6%E5%B1%8B+%E6%97%A5%E5%BC%8F%E6%96%99%E7%90%86/data=!4m7!3m6!1s0x3442a94894c7c60d:0xee2cf49590e19cd7!8m2!3d25.0703399!4d121.518217!16s%2Fg%2F1tgyz8rv!19sChIJDcbHlEipQjQR15zhkJX0LO4?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%B0%8F%E9%B3%B3%E7%B1%B3%E7%B2%89%E6%B9%AF/data=!4m7!3m6!1s0x3442a94848ca4b7d:0x1a17335527dfb78a!8m2!3d25.0716765!4d121.515944!16s%2Fg%2F11ddx78f5w!19sChIJfUvKSEipQjQRirffJ1UzFxo?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E8%A1%97%E7%B1%B3%E7%B2%89%E6%B9%AF/data=!4m7!3m6!1s0x3442a947c34aac65:0xd45cda3a024f7b39!8m2!3d25.0688559!4d121.515858!16s%2Fg%2F11c3x9nxxh!19sChIJZaxKw0epQjQROXtPAjraXNQ?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%A4%A7%E9%BE%8D%E5%B3%92%E9%98%BF%E5%AC%A4%E9%BA%B5%E7%B7%9A%E5%BA%97/data=!4m7!3m6!1s0x3442a949faa99f79:0xcfd46b7a94eff7f7!8m2!3d25.0744343!4d121.5148384!16s%2Fg%2F11cnc2t71p!19sChIJeZ-p-kmpQjQR9_fvlHpr1M8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E8%A3%B4%E8%A8%98%E5%86%B7%E5%87%8D%E6%89%8B%E5%B7%A5%E6%B0%B4%E9%A4%83/data=!4m7!3m6!1s0x3442a9483b5ae375:0xad122d2869aac5a!8m2!3d25.0715278!4d121.5149056!16s%2Fg%2F1w10_x2b!19sChIJdeNaO0ipQjQRWqyahtIi0Qo?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E4%B8%8B%E8%B7%AF%E4%BB%94%E7%B1%B3%E8%8B%94%E7%9B%AE/data=!4m7!3m6!1s0x3442a946542c1e9f:0xa35e2b2b6ae1dda6!8m2!3d25.0667713!4d121.5173061!16s%2Fg%2F1jglzj26q!19sChIJnx4sVEapQjQRpt3haisrXqM?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E5%85%AB%E6%96%B9%E9%9B%B2%E9%9B%86+(%E5%8F%B0%E5%8C%97%E5%A4%A7%E9%BE%8D%E5%BA%97)/data=!4m7!3m6!1s0x3442a96b4768077d:0xa5678ddc85a88abf!8m2!3d25.0717907!4d121.5159395!16s%2Fg%2F1tfthxkb!19sChIJfQdoR2upQjQRv4qohdyNZ6U?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/D.G.+Caf%C3%A9/data=!4m7!3m6!1s0x3442a93e7275398b:0xbb53941fa6e594f6!8m2!3d25.0622092!4d121.5091733!16s%2Fg%2F11by_xc0cw!19sChIJizl1cj6pQjQR9pTlph-UU7s?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%94%9C%E5%9C%A8%E5%BF%83%E5%9C%B0%E7%93%9CQQ%E7%90%83/data=!4m7!3m6!1s0x3442a9eb09c2c235:0xef15613dc780fd86!8m2!3d25.0717389!4d121.5159681!16s%2Fg%2F11f5f9qy5z!19sChIJNcLCCeupQjQRhv2Axz1hFe8?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E7%89%9B%E5%A5%B6%E5%A5%BD%E6%A3%92/data=!4m7!3m6!1s0x3442a94845d3b7f7:0x3bbceec3abd016ce!8m2!3d25.0714146!4d121.5159795!16s%2Fg%2F11gh04cp56!19sChIJ97fTRUipQjQRzhbQq8PuvDs?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/%E9%8D%8B%E6%88%B0%E3%81%AE%E7%89%A9-%E5%A4%A7%E9%BE%8D%E5%BA%97/data=!4m7!3m6!1s0x3442a90bf1c05289:0x60ecfc93057129c3!8m2!3d25.0714653!4d121.5157721!16s%2Fg%2F11nt5gjbg4!19sChIJiVLA8QupQjQRwylxBZP87GA?authuser=0&hl=zh-TW&rclk=1",
    "https://www.google.com.tw/maps/place/KONMUSU%E6%97%A5%E5%BC%8F%E9%A3%AF%E7%B3%B0%EF%BC%88%E6%90%AC%E5%AE%B6%E4%B8%AD%E5%8D%81%E4%B8%80%E6%9C%88%E5%BA%95%E5%86%8D%E9%96%8B%EF%BC%89/data=!4m7!3m6!1s0x3442af7618392e27:0xa29ca7e464584a82!8m2!3d25.0672248!4d121.5167164!16s%2Fg%2F11jlqrq9jf!19sChIJJy45GHavQjQRgkpYZOSnnKI?authuser=0&hl=zh-TW&rclk=1"
]

In [ ]:
def initialize_chrome(_from="facebook",retry=0):
    global driver 
    try:
        print("Initializing chromedriver.")
        options = Options()
        options.add_argument("--start-maximized")
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")
        options.add_argument('--log-level=3')
        driver = uc.Chrome(options=options)
        time.sleep(2)
        return True
    except Exception as e:
        print(e)
        pass

def parse_description(description_tag):
    description_text = h.handle(str(description_tag)) 
    return description_text

## 店家詳細資訊

In [41]:

def place_info():
    initialize_chrome()
    place_data = []
    for i in url_list:
        driver.get(i)
        response = driver.page_source
        soup = BeautifulSoup(response, 'html.parser')


        rating_and_review_div = soup.find('div',{'class':'k7jAl lJ3Kh miFGmb'})
        # print(rating_and_review_div)


        # 店名
        alias = rating_and_review_div.find(class_="fontHeadlineLarge").text.strip()
    #     print('店名: ' + alias)

        # 評論數
        total_reviews = rating_and_review_div.find('span',{'aria-label':re.compile("^\d.+ 則評論")})
        if total_reviews:
            total_reviews = total_reviews.get('aria-label','').strip() 
            total_reviews = total_reviews.removesuffix('則評論').strip()
        if not total_reviews:
            total_reviews = ""
    #     print('評論數: ' + total_reviews)

        # 星級
        total_rating = rating_and_review_div.find('span',{'aria-label':re.compile("^\s+?\d+\.\d+ 星級")})
        if total_rating:
            total_rating = total_rating.get('aria-label','').strip() 
            total_rating = total_rating.removesuffix('星級').strip()
        if not total_rating:
            total_rating = ""
    #     print('星級: ' + total_rating)

        # 標籤
        category = rating_and_review_div.find('button',{'jsaction':re.compile("pane.rating.category")})
        if category:
            category = category.getText()
        if not category:
            category = ""
    #     print('標籤: ' + category)

        # 價位
        price = rating_and_review_div.find('span',{'aria-label':re.compile("^價格")})
        if price:
            price = price.getText()
        if not price:
            price = ""
    #     print('價位: ' + price)

        # 地址
        address = rating_and_review_div.find('button',{'aria-label':re.compile("^地址\:")})
        if address:
            address = address.get('aria-label','').strip() 
            address = address.removeprefix('地址:')      
        if not address:
            address = ""
    #     print('地址: ' + address)

        # 電話
        phone = rating_and_review_div.find('button',{'aria-label':re.compile("^電話號碼\:")})
        if phone:
            phone = phone.get('aria-label','').strip()    
            phone = phone.removeprefix('電話號碼:')
        if not phone:
            phone = ""
    #     print('電話: ' + phone)

        # 營業時間
        opening_hours = rating_and_review_div.find('div',{'aria-label':re.compile("^星期")})
        if opening_hours:
            opening_hours = opening_hours.get('aria-label','')   
        if not opening_hours:
            opening_hours = ""
    #     print('營業時間: ' + opening_hours)

        # 網站
        website = rating_and_review_div.find('a',{'aria-label':re.compile("^網站\:")})
        if website:
            website = website.get('href')   
        if not website:
            website = ""
    #     print('網站: ' + website)


        
        info_dict = {}        
        info_dict['店名'] = alias
        info_dict['評論數'] =  total_reviews             
        info_dict["星級"] = total_rating
        info_dict["標籤"] = category
        info_dict['價位'] = price
        info_dict['地址'] =  address             
        info_dict["電話"] = phone
        info_dict["營業時間"] = opening_hours
        info_dict["網站"] = website

        place_data.append(info_dict)
        print('完成店名: ' + alias)
    place_info_header = list(place_data[0].keys())
    place_info_df = pd.DataFrame.from_records(place_data,columns=place_info_header)
    place_info_df.to_csv(f'./place_info.csv')
        

In [42]:
time1 = time.time()
place_info()
print(f'執行總花費時間: {time.time() - time1}')

Initializing chromedriver.
完成店名: 魚の棧(魚之棧)
完成店名: 蕃茄妹小吃店
完成店名: 阿仁炒飯
完成店名: •壹柒食堂•
完成店名: 巴八八小吃店
完成店名: 鐵板城現炒（大龍店）
完成店名: 李記雞窩/大同美食/大同雞肉/大同便當/大同小吃/大同餐廳/大同放山雞
完成店名: 大牛牛排館
完成店名: 大龍街香菇肉粥
完成店名: 大龍廣東粥
完成店名: 大雄香雞翅
完成店名: 大龍峒肉羹-米粉湯
完成店名: 老地方牛肉麵食館
完成店名: 炸G行家-大龍峒店（炸雞行家.美式炸雞.平價美食）
完成店名: 三大牛肉麵
完成店名: 開燈找餐
完成店名: 大高雄燒肉飯
完成店名: 蕼蒔荳留
完成店名: 大龍沙威瑪
完成店名: 第一土鵝肉
完成店名: 小林麵食店
完成店名: 湯竺麵食館
完成店名: 大龍城自助餐
完成店名: 台南東山鴨頭
完成店名: 大肥羴燒烤羊肉轉型（古早味麵店）
完成店名: 簡家大龍峒肉圓
完成店名: 大眾海鮮粥
完成店名: 雙連 文記花枝焿
完成店名: 老蔡水煎包 大龍創始店
完成店名: 吉祥素食
完成店名: 重慶豆漿炸蛋餅
完成店名: 272早午餐
完成店名: 吉利 牛肉麵.水餃
完成店名: 大龍峒魚丸
完成店名: 叁道街手工蚵仔麵線
完成店名: 三媽臭臭鍋
完成店名: 大龍街271號黃金地瓜QQ球
完成店名: 金好呷好吃雞肉飯
完成店名: 新口味鹽酥雞
完成店名: 全龍健康素食
完成店名: 尚品香早餐店（大龍店）
完成店名: 小里子海鮮
完成店名: 雲祥麵食
完成店名: 京品廣東粥
完成店名: 潮味決‧湯滷專門店 台北大龍分社
完成店名: 唐山婆古早味炸肉
完成店名: 大龍早午餐
完成店名: 大龍峒熊會長油飯
完成店名: 大龍街夜市-廟口的飯
完成店名: 果然文創
完成店名: 大龍峒一生懸命黑白切
完成店名: 梅滿美食
完成店名: 讚賞麵屋
完成店名: 弘爺漢堡
完成店名: 德龍飲食店
完成店名: 寶師傅私房牛肉麵
完成店名: 第一茗狀元糕
完成店名: 萬世香小籠包豆漿店
完成店名: 老麵店
完成店名: Q Burger 大同大龍店
完成店名: 圓環魯肉飯肉羹/大同區小吃 大同區美食 大同區滷肉飯 圓山美食 圓山小吃 圓山滷肉飯
完成店名: 油條花生湯
完成店名: 5味臭臭鍋 大龍店
完成店名: 小漢堡早餐
完成店名:

## 評論爬取

In [10]:

h = html2text.HTML2Text()
h.ignore_links = True

# 下一頁review內容的token(寫進url)
next_page_token = ""

# 設定biz_id(寫進url)
biz_id = re.search(r"1s(0.*?\:.*?)[^a-zA-Z\d\s:]",url_list[0]) # 寫成迴圈的時候要改一下
if not biz_id:
    print("Not a valid url.")
biz_id = biz_id.groups()[0]

reviewData = []

while True:
    
    print('已下載', len(reviewData))
    url = f'https://www.google.com/async/reviewSort?yv=3&async=feature_id:{biz_id},review_source:All%20reviews,sort_by:newestFirst,is_owner:false,filter_text:,associated_topic:,next_page_token:{next_page_token},_pms:s,_fmt:json'
    response = requests.get(url)

    response = response.text.removeprefix(")]}'")
    json_data = json.loads(response)["localReviewsProto"]
    review_data = json_data["other_user_review"]

    for result in review_data:
        rating = result['star_rating']['value']
        profile_pic = result['profile_photo_url']
        profile_pic = re.sub('=s(\d+)-','=s180-',profile_pic)
        reviewer_name = result['author_real_name']
        review_date = result['publish_date']['localized_date']      
        if result.get('review_text'):
            review_text = parse_description(result['review_text']['full_html'])
        else:
            review_text = ""

        info_dict = {}        
        info_dict['author'] = reviewer_name
        info_dict['rating'] =  rating             
        info_dict["date"] = review_date
        info_dict["text"] = review_text

        reviewData.append(info_dict)

    next_page_token = json_data.get('next_page_token','').strip()
    if not next_page_token:
        break
        
reviews_info_header = list(reviewData[0].keys())
reviews_info_df = pd.DataFrame.from_records(reviewData,columns=reviews_info_header)
reviews_info_df.to_csv(f'./{alias}_review.csv')

已下載 0
已下載 10
已下載 20
已下載 30
已下載 40
已下載 50
已下載 60
已下載 70
已下載 80
已下載 90
已下載 100
已下載 110
已下載 120
已下載 130
已下載 140
已下載 150
已下載 160
已下載 170
已下載 180
已下載 190
已下載 200
已下載 210
已下載 220
已下載 230
已下載 240
已下載 250
已下載 260
已下載 270
已下載 280
已下載 290
已下載 300
已下載 310
已下載 320
已下載 330
已下載 340
已下載 350
已下載 360
已下載 370
已下載 380
已下載 390
已下載 400
已下載 410
已下載 420
已下載 430
已下載 440
已下載 450
已下載 460
已下載 470
已下載 480
已下載 490
已下載 500
已下載 510
已下載 520
已下載 530
已下載 540
已下載 550
已下載 560
已下載 570
已下載 580
已下載 590
已下載 600
已下載 610
已下載 620
已下載 630
已下載 640
已下載 650
已下載 660
已下載 670
已下載 680
已下載 690
已下載 700
已下載 710
已下載 720
已下載 730
已下載 740
已下載 750
已下載 760
已下載 770
已下載 780
已下載 790
已下載 800
已下載 810
已下載 820
已下載 830
已下載 840
已下載 850
已下載 860
已下載 870
已下載 880
已下載 890
已下載 900
已下載 910
已下載 920
已下載 930
已下載 940
已下載 950
已下載 960
已下載 970
已下載 980
已下載 990
已下載 1000
已下載 1010
已下載 1020
已下載 1030
已下載 1040
已下載 1050
已下載 1060
已下載 1070
已下載 1080
已下載 1090
已下載 1100
已下載 1110
已下載 1120
已下載 1130
已下載 1140
已下載 1150
已下載 1160
已下載 1170
已下載 1180
已下載 1190
已下載 1200
已下載 1210
已下載 1220
已下載 